<a href="https://colab.research.google.com/github/egormarusev/graduate_work/blob/dev/cleaning_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=ecbcad4812e94a1976d1370d5ea40c71df7f0c9a10bfa2e9cf5333216428459c
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


In [ ]:
import ast
import emoji
import pandas as pd
import numpy as np
import os

In [ ]:
# full list of emojis
emojis = ['☃️','⚡️','✍️','❤️','❤️\u200d🔥','🆒','🌚','🌭','🍌','🍓','🍾','🎃','🎄','🎅','🎉','🏆','🐳','👀',
      '👌','👍','👎','👏','👨\u200d💻','👻','💅','💊','💋','💔','💘','💩','💯','🔥','🕊','🖕','🗿',
      '😁','😂','😇','😈','😍','😐','😒','😘','😢','😨','😭','😱','😳','😴','🙈','🙉','🙊','🙏','🤓',
      '🤔','🤗','🤝','🤡','🤣','🤨','🤩','🤪','🤬','🤮','🤯','🥰','🥱','🥴','🦄','\U0001fae1']

# create df with text the description of emojis
emojis_text = []
for i in emojis:
    emojis_text.append(emoji.demojize(i))


def parse_media(x):
    if pd.isnull(x):
        return x
    else:
        if 'photo' in x:
            try:
                return x['photo']['_']
            except:
                print(x)
        elif 'webpage' in x:
            return x['webpage']['_']
        
        
def parse_reactions(row):
    x = row['reactions']
    if not pd.isnull(x):
        res = x['results']
        for obj in res:
            emoticon = emoji.demojize(obj['reaction']['emoticon'])
            count = obj['count']
            df.loc[df.id == row.id, emoticon] = count


def parse_replies(data):
    data = ast.literal_eval(data)
    return data['replies']


def parse_fwd_from(data):
    data = ast.literal_eval(data)
    from_id = data['from_id']
    if pd.isnull(from_id):
        return np.nan
    if 'channel_id' in from_id:
        id = from_id['channel_id']
        if id in channels_id:
            return channels_id[id]
        else:
            return id
    else:
        return from_id['user_id']


def cleaning_data(data):
    # # drop unnecessary columns
    # data.drop(['_', 
    #           'action',
    #           'ttl_period',
    #           'restriction_reason',
    #           'grouped_id',
    #           'post_author',
    #           'entities',
    #           'via_bot_id',
    #           'from_id',
    #           'noforwards',
    #           'pinned',
    #           'from_scheduled',
    #           'legacy',
    #           'media_unread',
    #           'reply_markup'
    #           'out'], axis=1, inplace=True)
    
    # # parse column peer_id
    # data = data.drop('peer_id', axis=1).assign(**pd.DataFrame(data.peer_id.values.tolist()))
    # data.drop('_', axis=1, inplace=True)
    
    # parse column reply_to
    data.reply_to = data.reply_to.apply(lambda x: {} if pd.isnull(x) else ast.literal_eval(x))
    data = data.drop('reply_to', axis=1).assign(**pd.DataFrame(data.reply_to.values.tolist()))
    data.drop(['_',
             'reply_to_scheduled',
             'forum_topic', 
             'reply_to_peer_id',
             'reply_to_top_id'], axis=1, inplace=True)
    
    # parse column media
    data.media = data.media.apply(lambda x: x if pd.isnull(x) else parse_media(ast.literal_eval(x)))
    
    # concat emojis df with main df
    emoji_df = pd.DataFrame(0, index=np.arange(len(data)), columns=emojis_text)
    data = pd.concat([data, emoji_df], axis=1)
    
    # parse reactions
    data.reactions = data.reactions.apply(lambda x: x if pd.isnull(x) else ast.literal_eval(x))
    data.apply(parse_reactions, axis=1)
    data.drop('reactions', axis=1, inplace=True)

    # parse replies
    df.replies = df.replies.apply(lambda x: x if pd.isnull(x) else parse_replies(x))

    # parse fwd_from
    df.fwd_from = df.fwd_from.apply(lambda x: x if pd.isnull(x) else parse_fwd_from(x))

In [ ]:
names_of_channels = ['bbcrussian', 'lentadnya', 'mash', 'mediazzzona',
                     'rbc_news', 'readovkanews', 'rian_ru', 'rt_russian',
                     'tass_agency', 'TJournal', 'truekpru', 'tvrain']

In [ ]:
subs = []
for name in names_of_channels:
  with open(f'/content/drive/MyDrive/datasets/news/subs/{name}.txt', 'r') as f:
    lines = f.readlines()
    date_arr = []
    subs_arr = []
    for i in range(len(lines)):
      if lines[i] == '            <div class="col-4 px-1 text-muted text-truncate">\n':
        date_arr.append(lines[i+1].split()[0])
        subs_arr.append(lines[i+3].split()[0])
  subs_df = pd.DataFrame({'date': date_arr, 'num_subs': subs_arr})
  subs_df['date']= pd.to_datetime(subs_df['date'], format='%d.%m.%y')
  subs.append(subs_df)

In [ ]:
df_channels = []

for i, name in enumerate(names_of_channels):
    tmp = pd.read_json(f'/content/drive/MyDrive/datasets/news/{name}.json')
    tmp.drop(['_', 'action', 'ttl_period', 'restriction_reason',
            'grouped_id', 'post_author', 'entities', 'via_bot_id', 'from_id',
            'noforwards', 'pinned', 'from_scheduled', 'legacy',
            'media_unread', 'reply_markup', 'out'], axis=1, inplace=True)
    tmp['channel_name'] = name
    tmp['only_date'] = tmp['date'].dt.date

    first_subs_day = subs[i].date.min().date()
    first_ind = tmp[tmp['only_date'] >= first_subs_day].index[-1]
    tmp = tmp.loc[:first_ind].reset_index(drop=True)
    tmp['only_date'] = pd.to_datetime(tmp['only_date']).dt.date
    subs[i]['date'] = subs[i]['date'].dt.date
    tmp = pd.merge(tmp, subs[i], left_on='only_date', right_on='date', how='left')
    df_channels.append(tmp)

df = pd.concat(df_channels)

/usr/local/lib/python3.8/dist-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


In [ ]:
print(df.shape)
df.head()

In [ ]:
df.loc[df.message.isna()]

In [ ]:
df.dropna(subset=['message'], inplace=True)  # удалили строки, где message пустой

In [ ]:
df.loc[df.message.isna()]

,id,peer_id,date_x,message,mentioned,silent,post,edit_hide,fwd_from,reply_to,media,views,forwards,replies,edit_date,reactions,channel_name,only_date,date_y,num_subs


In [ ]:
df.loc[df.num_subs.isna()].head()

In [ ]:
df.num_subs.isna().sum()

2192

In [ ]:
df.dropna(subset=['num_subs'], inplace=True)  # удалили строки, где num_subs пустой
df.reset_index(drop=True, inplace=True)

In [ ]:
df.num_subs.isna().sum()

0

In [ ]:
df.head()

In [ ]:
df['hour'] = df.date_x.dt.hour
df['minute'] = df.date_x.dt.minute
df['seconds'] = df.date_x.dt.second

In [ ]:
df.rename(columns={'date_x': 'date'}, inplace=True)

In [ ]:
df.drop(['peer_id', 'date_y'], axis=1, inplace=True)

In [ ]:
df['day'] = df.date.dt.day
df['month'] = df.date.dt.month
df['year'] = df.date.dt.year

In [ ]:
df.columns

Index(['id', 'date', 'message', 'mentioned', 'silent', 'post', 'edit_hide',
       'fwd_from', 'reply_to', 'media', 'views', 'forwards', 'replies',
       'edit_date', 'reactions', 'channel_name', 'only_date', 'num_subs',
       'hour', 'minute', 'seconds'],
      dtype='object')

In [ ]:
df.to_csv('full_data_.csv')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/datasets/news/full_data_.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (8,13,15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
cleaning_data(df)

{'_': 'MessageMediaInvoice', 'title': 'Поддержать Дождь', 'description': 'Мы продолжаем свою работу благодаря вашей поддержке. Для удобства мы настроили возможность стать спонсором канала и напрямую в телеграме', 'currency': 'RUB', 'total_amount': 30000, 'start_param': '', 'shipping_address_requested': False, 'test': False, 'photo': None, 'receipt_msg_id': None, 'extended_media': None}
{'_': 'MessageMediaInvoice', 'title': 'Поддержать Дождь', 'description': 'Мы продолжаем свою работу благодаря вашей поддержке. Для удобства мы настроили возможность стать спонсором канала и напрямую в телеграме:', 'currency': 'RUB', 'total_amount': 30000, 'start_param': '', 'shipping_address_requested': False, 'test': False, 'photo': None, 'receipt_msg_id': None, 'extended_media': None}


In [ ]:
df.head()

,Unnamed: 0,id,date,message,mentioned,silent,post,edit_hide,fwd_from,reply_to,...,:man_technologist:,:moai:,:nail_polish:,:nerd_face:,:unicorn:,:smiling_face_with_halo:,:handshake:,:hear-no-evil_monkey:,:face_blowing_a_kiss:,:Christmas_tree:
0,0,41356,2023-02-04 17:44:01+00:00,Украина приняла на вооружение цифровую систему...,False,False,True,0.0,NaN,{},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,41355,2023-02-04 17:30:08+00:00,Daily Telegraph: британские власти планируют о...,False,False,True,0.0,NaN,{},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,41354,2023-02-04 16:33:01+00:00,Село Красное в Белгородской области попало под...,False,False,True,0.0,NaN,{},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,41353,2023-02-04 16:18:25+00:00,Главное к этому часу. Идет 346-й день войны\n\...,False,False,True,0.0,NaN,{},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,41351,2023-02-04 15:32:52+00:00,Глава офиса президента Украины Андрей Ермак со...,False,False,True,0.0,NaN,{},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
df.loc[~df.reply_to.isna()]

,id,date,message,mentioned,silent,post,edit_hide,fwd_from,reply_to,media,...,:man_technologist:,:moai:,:nail_polish:,:nerd_face:,:unicorn:,:smiling_face_with_halo:,:handshake:,:hear-no-evil_monkey:,:face_blowing_a_kiss:,:Christmas_tree:
0,41356,2023-02-04 17:44:01+00:00,Украина приняла на вооружение цифровую систему...,False,False,True,0.0,NaN,{},NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,41355,2023-02-04 17:30:08+00:00,Daily Telegraph: британские власти планируют о...,False,False,True,0.0,NaN,{},NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,41354,2023-02-04 16:33:01+00:00,Село Красное в Белгородской области попало под...,False,False,True,0.0,NaN,{},NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,41353,2023-02-04 16:18:25+00:00,Главное к этому часу. Идет 346-й день войны\n\...,False,False,True,0.0,NaN,{},NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,41351,2023-02-04 15:32:52+00:00,Глава офиса президента Украины Андрей Ермак со...,False,False,True,0.0,NaN,{},"{'_': 'MessageMediaWebPage', 'webpage': {'_': ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999994,2276,2017-05-09 08:55:09+00:00,На Красной площади в Москве завершился парад П...,False,False,True,0.0,NaN,{},"{'_': 'MessageMediaWebPage', 'webpage': {'_': ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
999995,2275,2017-05-09 07:44:23+00:00,В параде впервые участвуют отряды «Юнармии» — ...,False,False,True,0.0,NaN,{},"{'_': 'MessageMediaWebPage', 'webpage': {'_': ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
999996,2274,2017-05-09 07:44:20+00:00,NaN,False,False,True,0.0,NaN,{},"{'_': 'MessageMediaPhoto', 'photo': {'_': 'Pho...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
999997,2272,2017-05-09 06:47:18+00:00,В 10:00 мы начинаем трансляцию парада Победы в...,False,False,True,0.0,NaN,{},"{'_': 'MessageMediaWebPage', 'webpage': {'_': ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.to_csv('full_parsed_data.csv', index=False)

In [ ]:
df.loc[~df.reactions.isna(), 'reactions'][49965]

"{'_': 'MessageReactions', 'results': [{'_': 'ReactionCount', 'reaction': {'_': 'ReactionEmoji', 'emoticon': '😁'}, 'count': 19, 'chosen_order': None}, {'_': 'ReactionCount', 'reaction': {'_': 'ReactionEmoji', 'emoticon': '🤡'}, 'count': 15, 'chosen_order': None}, {'_': 'ReactionCount', 'reaction': {'_': 'ReactionEmoji', 'emoticon': '👎'}, 'count': 10, 'chosen_order': None}], 'min': False, 'can_see_list': False, 'recent_reactions': []}"

In [ ]:
df.columns

Index(['id', 'date', 'message', 'mentioned', 'silent', 'post', 'edit_hide',
       'fwd_from', 'reply_to', 'media', 'views', 'forwards', 'replies',
       'edit_date', 'reactions', 'channel_name', 'only_date', 'num_subs',
       'hour', 'minute', 'seconds', ':beaming_face_with_smiling_eyes:',
       ':clown_face:', ':thumbs_down:', ':thumbs_up:', ':fire:',
       ':face_with_symbols_on_mouth:', ':red_heart:',
       ':face_screaming_in_fear:', ':smiling_face_with_horns:',
       ':smiling_face_with_heart-eyes:', ':clapping_hands:', ':crying_face:',
       ':face_vomiting:', ':rolling_on_the_floor_laughing:', ':thinking_face:',
       ':pile_of_poo:', ':strawberry:', ':dove:', ':party_popper:',
       ':high_voltage:', ':smiling_face_with_hearts:', ':trophy:', ':hot_dog:',
       ':neutral_face:', ':exploding_head:', ':new_moon_face:',
       ':middle_finger:', ':face_with_raised_eyebrow:', ':heart_on_fire:',
       ':woozy_face:', ':broken_heart:', ':star-struck:', ':folded_hands:',
  

In [ ]:
df.loc[df['reply_to'] != {}, 'reply_to'][41]

{'_': 'MessageReplyHeader',
 'reply_to_msg_id': 41308,
 'reply_to_scheduled': False,
 'forum_topic': False,
 'reply_to_peer_id': None,
 'reply_to_top_id': None}

In [ ]:
df.loc[~df.media.isna(), 'media']

4         {'_': 'MessageMediaWebPage', 'webpage': {'_': ...
6         {'_': 'MessageMediaWebPage', 'webpage': {'_': ...
7         {'_': 'MessageMediaWebPage', 'webpage': {'_': ...
16        {'_': 'MessageMediaWebPage', 'webpage': {'_': ...
17        {'_': 'MessageMediaWebPage', 'webpage': {'_': ...
                                ...                        
999994    {'_': 'MessageMediaWebPage', 'webpage': {'_': ...
999995    {'_': 'MessageMediaWebPage', 'webpage': {'_': ...
999996    {'_': 'MessageMediaPhoto', 'photo': {'_': 'Pho...
999997    {'_': 'MessageMediaWebPage', 'webpage': {'_': ...
999998    {'_': 'MessageMediaPhoto', 'photo': {'_': 'Pho...
Name: media, Length: 495260, dtype: object

In [ ]:
!mv /content/drive/MyDrive/full_parsed_data.csv /content/drive/MyDrive/datasets/news/

In [ ]:
!ls -la /content/drive/MyDrive/datasets/news/

total 10410386
-rw------- 1 root root  306591634 Feb  4 17:58 bbcrussian.json
-rw------- 1 root root 2540721789 Feb 20 16:41 full_data.csv
-rw------- 1 root root 2490730153 Feb 21 22:45 full_data_.csv
-rw------- 1 root root 2582141196 Feb 22 18:36 full_parsed_data.csv
-rw------- 1 root root  176755797 Feb  8 15:36 lentadnya.json
-rw------- 1 root root   98761173 Feb  8 17:01 mash.json
-rw------- 1 root root  112545974 Feb  8 16:08 mediazzzona.json
-rw------- 1 root root  390104882 Feb  4 18:11 rbc_news.json
-rw------- 1 root root  188550787 Feb  4 16:51 readovkanews.json
-rw------- 1 root root  280406439 Feb  4 17:24 rian_ru.json
-rw------- 1 root root  447213466 Feb  4 17:51 rt_russian.json
drwx------ 2 root root       4096 Feb 21 21:34 subs
-rw------- 1 root root  380789083 Feb  8 16:49 tass_agency.json
-rw------- 1 root root  132870054 Feb  8 16:18 TJournal.json
-rw------- 1 root root  320205711 Feb  8 16:06 truekpru.json
-rw------- 1 root root  211839043 Feb  8 15:46 tvrain.json


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/datasets/news/full_parsed_data.csv')
df.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (7,12,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,date,message,mentioned,silent,post,edit_hide,fwd_from,reply_to,media,...,:man_technologist:,:moai:,:nail_polish:,:nerd_face:,:unicorn:,:smiling_face_with_halo:,:handshake:,:hear-no-evil_monkey:,:face_blowing_a_kiss:,:Christmas_tree:
0,41356,2023-02-04 17:44:01+00:00,Украина приняла на вооружение цифровую систему...,False,False,True,0.0,NaN,{},NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,41355,2023-02-04 17:30:08+00:00,Daily Telegraph: британские власти планируют о...,False,False,True,0.0,NaN,{},NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,41354,2023-02-04 16:33:01+00:00,Село Красное в Белгородской области попало под...,False,False,True,0.0,NaN,{},NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,41353,2023-02-04 16:18:25+00:00,Главное к этому часу. Идет 346-й день войны\n\...,False,False,True,0.0,NaN,{},NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,41351,2023-02-04 15:32:52+00:00,Глава офиса президента Украины Андрей Ермак со...,False,False,True,0.0,NaN,{},"{'_': 'MessageMediaWebPage', 'webpage': {'_': ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.reply_to = df.reply_to.apply(lambda x: {} if pd.isnull(x) else ast.literal_eval(x))
df = df.drop('reply_to', axis=1).assign(**pd.DataFrame(df.reply_to.values.tolist()))
df.drop(['_',
            'reply_to_scheduled',
            'forum_topic', 
            'reply_to_peer_id',
            'reply_to_top_id'], axis=1, inplace=True)

In [ ]:
df.loc[~df.reply_to_msg_id.isna(), 'reply_to_msg_id']

41        41308.0
44        41308.0
64        41289.0
73        41267.0
77        41276.0
           ...   
996654     5800.0
996828     5615.0
996832     5611.0
997057     5375.0
999166     3154.0
Name: reply_to_msg_id, Length: 113999, dtype: float64

In [ ]:
df.loc[~df.media.isna(), 'media']

4              WebPage
6              WebPage
7              WebPage
16             WebPage
17             WebPage
              ...     
999994    WebPageEmpty
999995         WebPage
999996           Photo
999997         WebPage
999998           Photo
Name: media, Length: 363018, dtype: object

In [ ]:
df.to_csv('/content/drive/MyDrive/datasets/news/news.csv')

In [ ]:
df.head()

,id,date,message,mentioned,silent,post,edit_hide,fwd_from,media,views,...,:moai:,:nail_polish:,:nerd_face:,:unicorn:,:smiling_face_with_halo:,:handshake:,:hear-no-evil_monkey:,:face_blowing_a_kiss:,:Christmas_tree:,reply_to_msg_id
0,41356,2023-02-04 17:44:01+00:00,Украина приняла на вооружение цифровую систему...,False,False,True,0.0,NaN,NaN,6355.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,41355,2023-02-04 17:30:08+00:00,Daily Telegraph: британские власти планируют о...,False,False,True,0.0,NaN,NaN,13775.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,41354,2023-02-04 16:33:01+00:00,Село Красное в Белгородской области попало под...,False,False,True,0.0,NaN,NaN,30614.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,41353,2023-02-04 16:18:25+00:00,Главное к этому часу. Идет 346-й день войны\n\...,False,False,True,0.0,NaN,NaN,34170.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,41351,2023-02-04 15:32:52+00:00,Глава офиса президента Украины Андрей Ермак со...,False,False,True,0.0,NaN,WebPage,43052.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
a = pd.read_csv('/content/drive/MyDrive/datasets/news/news.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (8,12,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
a.shape

(999999, 92)

In [ ]:
a.isna().sum()

Unnamed: 0                    0
id                            0
date                          0
message                   81540
mentioned                     0
                          ...  
:handshake:              999841
:hear-no-evil_monkey:    999991
:face_blowing_a_kiss:    999991
:Christmas_tree:         999797
reply_to_msg_id          886000
Length: 92, dtype: int64

In [ ]:
a.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
a.to_csv('/content/drive/MyDrive/datasets/news/news.csv', index=False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/datasets/news/news.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (7,11,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df.head(1)

,id,date,message,mentioned,silent,post,edit_hide,fwd_from,media,views,...,:moai:,:nail_polish:,:nerd_face:,:unicorn:,:smiling_face_with_halo:,:handshake:,:hear-no-evil_monkey:,:face_blowing_a_kiss:,:Christmas_tree:,reply_to_msg_id
0,41356,2023-02-04 17:44:01+00:00,Украина приняла на вооружение цифровую систему...,False,False,True,0.0,NaN,NaN,6355.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.loc[~df.replies.isna(), 'replies']

168694    42.0
168695    10.0
168696    14.0
168697    52.0
168698    44.0
          ... 
890649     0.0
890725     0.0
891787     0.0
892076     0.0
892101     0.0
Name: replies, Length: 172167, dtype: float64

In [ ]:
df.loc[~df.fwd_from.isna(), 'fwd_from']

31941     1.038403e+09
33374     1.038403e+09
34223     1.106076e+09
34255     1.106076e+09
34274     1.106076e+09
              ...     
997934    1.118579e+09
998233    1.000058e+09
999098    3.973676e+08
999115    3.973676e+08
999167    3.973676e+08
Name: fwd_from, Length: 34244, dtype: float64

In [ ]:
channels_id = {41356: 'bbcrussian',
               70375: 'lentadnya',
               10626: 'mediazzzona',
               67888: 'rbc_news',
               51976: 'readovkanews',
               193092: 'rian_ru',
               145268: 'rt_russian',
               179941: 'tass_agency',
               58374: 'TJournal',
               108377: 'truekpru',
               62601: 'tvrain'}

In [ ]:
df.drop('reactions', axis=1, inplace=True)

In [ ]:
df.head()

,id,date,message,mentioned,silent,post,edit_hide,fwd_from,media,views,...,:moai:,:nail_polish:,:nerd_face:,:unicorn:,:smiling_face_with_halo:,:handshake:,:hear-no-evil_monkey:,:face_blowing_a_kiss:,:Christmas_tree:,reply_to_msg_id
0,41356,2023-02-04 17:44:01+00:00,Украина приняла на вооружение цифровую систему...,False,False,True,0.0,NaN,NaN,6355.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,41355,2023-02-04 17:30:08+00:00,Daily Telegraph: британские власти планируют о...,False,False,True,0.0,NaN,NaN,13775.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,41354,2023-02-04 16:33:01+00:00,Село Красное в Белгородской области попало под...,False,False,True,0.0,NaN,NaN,30614.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,41353,2023-02-04 16:18:25+00:00,Главное к этому часу. Идет 346-й день войны\n\...,False,False,True,0.0,NaN,NaN,34170.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,41351,2023-02-04 15:32:52+00:00,Глава офиса президента Украины Андрей Ермак со...,False,False,True,0.0,NaN,WebPage,43052.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.to_csv('/content/drive/MyDrive/datasets/news/news.csv', index=False)